In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Загрузка словаря из файла ===
def load_dictionary(file_path):
    dictionary = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                dictionary[key.strip().lower()] = value.strip()
    return dictionary

# Загружаем словарь из файла
food_dict = load_dictionary('global-street-food.txt')

# === Создаем vocab на основе слов из словаря ===
words = set()
for key, value in food_dict.items():
    words.update(key.split())
    words.update(value.split())

vocab = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
for i, word in enumerate(words, start=3):
    vocab[word] = i
inv_vocab = {i: w for w, i in vocab.items()}
vocab_size = len(vocab)
embedding_dim = 8
hidden_size = 16
max_len = 10 

# Модель
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        _, h = self.rnn(x)
        return h

class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, h = self.rnn(x, h)
        out = self.fc(out)
        return out, h

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        h = self.encoder(src)
        out, _ = self.decoder(tgt, h)
        return out

encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
model = Seq2Seq(encoder, decoder).to(device)
# Генерация ответа
def respond_to_input(user_input):
    # Сначала проверяем точное совпадение в словаре
    user_input_lower = user_input.lower()
    if user_input_lower in food_dict:
        return f"В состав этого блюда входит: {food_dict[user_input_lower]}"
    
    # Загружаем список блюд из файла
    with open('dishes_names.txt', 'r', encoding='utf-8') as file:
        dishes = [line.strip() for line in file if line.strip()]
    
    # Формируем полный список блюд
    if dishes:
        dishes_list = "\n".join(f"- {dish}" for dish in dishes)  # Все блюда
        return (f"Я не знаю такого блюда.\n\n"
                f"Доступные блюда:\n{dishes_list}\n\n"
                f"Всего блюд: {len(dishes)}")
    else:
        return "Я не знаю такого блюда. База блюд пуста."

# Чат-бот
model.eval()
with torch.no_grad():
    print("Чат-бот о уличной еде. Напишите название блюда, состав которого вы хотите узнать, или 'выход' для завершения.")
    while True:
        user_input = input("Вы: ").strip()
        if user_input.lower() in {"выход", "exit", "quit"}:
            print("Бот: До свидания! Приятного аппетита!")
            break

        
        response = respond_to_input(user_input)
        print("Бот:", response)

Чат-бот о уличной еде. Напишите название блюда, состав которого вы хотите узнать, или 'выход' для завершения.


Вы:  hot dog


Бот: В состав этого блюда входит: Sausage, Bun, Mustard, Ketchup


Вы:  mause


Бот: Я не знаю такого блюда.

Доступные блюда:
- Takoyaki
- Vada Pav
- Funnel Cake
- Kathi Roll
- Pad Thai
- Pani Puri
- Taiyaki
- Gözleme
- Kokoreç
- Roti Sai Mai
- Yakisoba
- Quesadilla
- Elote
- Hot Dog
- Tacos al Pastor
- Simit
- Moo Ping
- Philly Cheesesteak
- Falafel
- Manakish
- Shawarma
- Okonomiyaki
- Yakisoba
- Bunny Chow
- Suya
- Koshari

Всего блюд: 26


Вы:  борщ


Бот: Я не знаю такого блюда.

Доступные блюда:
- Takoyaki
- Vada Pav
- Funnel Cake
- Kathi Roll
- Pad Thai
- Pani Puri
- Taiyaki
- Gözleme
- Kokoreç
- Roti Sai Mai
- Yakisoba
- Quesadilla
- Elote
- Hot Dog
- Tacos al Pastor
- Simit
- Moo Ping
- Philly Cheesesteak
- Falafel
- Manakish
- Shawarma
- Okonomiyaki
- Yakisoba
- Bunny Chow
- Suya
- Koshari

Всего блюд: 26


Вы:  выход


Бот: До свидания! Приятного аппетита!
